### The benchmark of SARCOS dataset

The performance of different algorithms on this dataset can be found in: https://github.com/Kaixhin/SARCOS

#### Ridge regression with fourier(polynomial()) feature: 2.111,  
NN feature not working.

In [ ]:
from jylearn.data.get_data import robot_inv_data
import numpy as np
import torch as th
device = "cuda" if th.cuda.is_available() else "cpu"

Loss = th.nn.MSELoss()
X_train, Y_train, X_val, Y_val, X_test, Y_test = robot_inv_data()

In [ ]:
X_train.shape

In [ ]:
X, Y = np.concatenate([X_train, X_val], axis=0), np.concatenate([Y_train, Y_val], axis=0)
from jylearn.feature.polynomial import PolynomialFT
from jylearn.feature.bellcurve import BellCurve
from jylearn.feature.fourier import FourierBases
from jylearn.parametric.ridge import RidgeReg

param1 = np.linspace(0.05, 0.99, 7)
param2 = np.arange(1, 7)

f1 = FourierBases(param1, param2)
f2 = PolynomialFT(2)

X_f = f1(f2(X))
X_test_f = f1(f2(X_test))
print("feature dim: ", X_f.shape[1])

In [ ]:
X_f_t, Y_t, X_test_f_t, Y_t_test = th.from_numpy(X_f).to(device), th.from_numpy(Y).to(device), th.from_numpy(X_test_f).to(device), th.from_numpy(Y_test).to(device)
rr = RidgeReg().fit(X_f_t, Y_t)
pred = rr.predict(X_test_f_t)
mse = Loss(pred, Y_t_test)
print(mse)

In [ ]:
X_train, Y_train, X_val, Y_val, X_test, Y_test =\
    th.from_numpy(X_train).to(device).double(), th.from_numpy(Y_train).to(device).double(), th.from_numpy(X_val).to(device).double(), \
    th.from_numpy(Y_val).to(device).double(), th.from_numpy(X_test).to(device).double(), th.from_numpy(Y_test).to(device).double()
    
from torch.optim import Adam
from torch.nn import MSELoss
from jylearn.parametric.mlp import MLP

#### Neural network: 1.469

In [ ]:
param = {"layer":4, "nodes":[21, 500, 500, 7], "actfunc":["ReLU", "ReLU", None], "batch":128, "lr":1e-3, "decay":0.}
net = MLP(param).to(device).double()
print(net)
Loss = MSELoss()
parameters = MLP.setParams(net, param["decay"])
optimizer = Adam(parameters, lr=param["lr"])
for _ in range(100):
    for i in range(len(X_train)//param["batch"]):
        optimizer.zero_grad()
        index = th.randperm(len(X_train))
        curr_index = index[i*param["batch"]:(i+1)*param["batch"]]
        X_b = X_train[curr_index]
        Y_b = Y_train[curr_index]
        pred = net(X_b)
        L = Loss(pred, Y_b)
        L.backward()
        optimizer.step()
    with th.no_grad():
        pred_val = net(X_val)
        L_val = Loss(pred_val, Y_val)
        print("Curr validation loss: ", L_val)
net.eval()
pred_test = net(X_test)
print(Loss(pred_test, Y_test))

#### subsample GPR with hyperparameters optimization: 2.887

In [ ]:
from jylearn.nonparametric.gpr import ExactGPR
from jylearn.kernel.kernels import White, RQK

In [ ]:
X_test, Y_test = X_test.double(), Y_test.double()
perm = th.randperm(X_train.size(0))
idx = perm[:5000]
X_train_, Y_train_ = X_train[idx].double(), Y_train[idx].double()

In [ ]:
l = np.ones((21,7)) * 30.
sigma = np.ones((7,)) * 1.
c = np.ones((7,)) * 1.

kernel = White(c=c, dim_in=21, dim_out=7) + RQK(l=l, sigma=sigma, alpha=sigma, dim_in=21, dim_out=7)
gpr = ExactGPR(kernel=kernel)
gpr.fit(X_train_, Y_train_, call_hyper_opt=True, lr=0.05, epoch=9000, optimizer_type="RMSPROP")

In [ ]:
pred = gpr.predict(X_test)
print(Loss(pred, Y_test))

del gpr, pred, kernel
th.cuda.empty_cache()

## Titsias' variational EM GPR: m=250, 2.8788

In [ ]:
from jylearn.nonparametric.vigpr import VariationalEMSparseGPR
from jylearn.kernel.kernels import White, RBF

l = np.ones([21, 1]) * 1.
sigma = np.ones([1,]) * 1.
c = np.ones([1,]) * 0.1

white_kernel = White(c=c, dim_in=21, dim_out=1)
kernel = RBF(sigma=sigma, l=l, dim_in=21, dim_out=1)

gpr = VariationalEMSparseGPR(kernel=kernel, white_kernle=white_kernel)
ind = gpr.fit(X_train, Y_train, m=20, subsetNum=100, lr=1e-2)

In [ ]:
pred = gpr.predict(X_test)
print(Loss(pred, Y_test))

del gpr, pred, kernel
th.cuda.empty_cache()